# Project Gutenberg Analysis: Data Cleaning Process

### Notebook 02: Excel Data Clean

## Overview
In this notebook, I will transition from Python to Excel for business-focused analysis of the Project Gutenberg dataset. I will document the Excel-based cleaning process for the dataset exported from the Python collection script. I'll focus on data validation, standardization, and preparation for analysis.

I'll start by showing the location of the files.

## File Information
   - Input file: `ProjectGutenberg_Analysis/data/processed/gutenberg_processed.csv` 
   - Working file: `ProjectGutenberg_Analysis/excel/gutenberg_cleaning_v1.xlsx`
   - Output file: - Filename: `ProjectGutenberg_Analysis/excel/gutenberg_cleaned_v1.xls`

### Step 1: Data Import and Initial Structure
1. Import Process

![Import process](../src/Screenshots/LoadCSV_PowerQuery.png)

Using PowerQuery, I import the data into an Excel spreadsheet.

![Import process](../src/Screenshots/LoadCSV_Result.png)


2. Initial Column Review
   - The current columns have the following data types:
   ```
   - Book_ID: Number
   - Title: Text
   - Author: Text
   - Publication_Date: Date
   - Language: Text
   - Download_Count: Number
   - Subjects: Text
   ```

### Step 2: Data Validation Checks
1. Check for Missing Values

![Conditional formatting](../src/Screenshots/Blanks_Conditional.png)

The first example of missing data was the title 'Beowulf' which famously has an unrecorded author (or lost to history). Rather than visually checking the 74,000 records, I will do a count like I would in Python.

![Blank counting](../src/Screenshots/Blanks_Count.png) 
![Blank counting](../src/Screenshots/Blanks_Count_Close.png) 

Some columns are completely full and three have missing values of authors, subjects and bookshelves at very small amounts. The available formats field is completely blank, which is disappointing for further analysis. The script looking for blanks in the Data Collection notebook suggested there were none.

2. Identify Duplicate Records

![Count of duplicates](../src/Screenshots/Duplicates_Count.png)

The dataset is remarkably clean with no duplicates (empty Series for Missing Values), which suggests high-quality data collection from the Gutenberg API. 

### Step 3: Column-by-Column Cleaning
Now, I will clean the columns one-by-one.

#### 1. ID Column
I am checking for:
- Duplicates:
```excel
# Create duplicate column check
Column: ID_Check
Formula: =COUNTIF(gutenberg_processed[id], [@id]) > 1
```

#### 2. Title Column
In any string of text, there are probably going to be the following issues:
- Extra spaces
- Inconsistent capitalization
- Special characters

```excel
# Create cleaned title column
Column: Clean_Title
Formula: =TRIM(PROPER([@title]))
```

#### 3. Authors Column
I can see that there are multiple issues for this column. there are
- Multiple authors separated by delimiters
- Inconsistent formatting
- Missing authors

```excel
# Create author status column
Column: Author_Status
Formula: =IF([@authors]="", "Missing",
            IF(ISNUMBER(SEARCH(";", [@authors])), "Multiple Authors", 
            "Single Author"))
```
![Author values](../src/Screenshots/Author_Status.png)
For the authors column, there are no missing authors and the majority of titles were written by single authors, with a smaller percentage written by multiple authors.

```excel
# Count number of authors
Column: Author_Count
Formula: =IF([@authors]="", 0,
            LEN([@authors])-LEN(SUBSTITUTE([@authors],";",""))+1)
```
![Author counts](../src/Screenshots/Author_Counts.png)

I am interested in the titles with multiple authors, so this pivot chart shows the titles and the author count.

![Author pivot table](../src/Screenshots/Author_Pivot.png)

Most of the entries with high author counts are anthologies or collections, technical reports with multiple contributors, literary compilations or scademic collaborations. Research shows that notable examples are:
- Atlantic Narratives (24 and 23 authors) - Short story collections
- Special report on diseases of cattle (14 authors) - Technical collaboration
- Various literary collections and compilations


#### 4. Languages Column
Check for:
- Invalid language codes
- Multiple languages

First, I created a list of ISO 639-1 codes as a table:

![Language codes](../src/Screenshots/Language_Codes.png)

Then, I added column to show if the title had multiple or single language versions:
```excel
# Create language status column
Column: Language_status
Formula: =IF([@languages]="", "Missing",
            IF(ISNUMBER(SEARCH(";", [@languages])), "Multiple Languages",
            "Single Language"))

And counted how many books had multiple languages. THe maximum languages was 3:
```excel
# Count number of languages
Column: Language_Count
Formula: =IF([@languages]="", 0,
            LEN([@languages])-LEN(SUBSTITUTE([@languages],";",""))+1)
```

![Language counts](../src/Screenshots/Language_Counts.png)

```excel
# Extract individual languages
Column: First_Language
Formula: =TRIM(LEFT([@languages],IFERROR(FIND(";",[@languages])-1,LEN([@languages]))))

Column: Second_Language
Formula: =IF(ISNUMBER(FIND(";",[@languages])),
            TRIM(MID([@languages],
                FIND(";",[@languages])+1,
                IFERROR(FIND(";",[@languages],FIND(";",[@languages])+1),LEN([@languages]))-FIND(";",[@languages])-1)),
            "")

Column: Third_Language
Formula: =IF(IFERROR(FIND(";",[@languages],FIND(";",[@languages])+1),0)>0,
           TRIM(MID([@languages],
                FIND(";",[@languages],FIND(";",[@languages])+1)+1,
                LEN([@languages])-FIND(";",[@languages],FIND(";",[@languages])+1))),
           "")

```
Once extracted, I will valid against the table of language codes.
```excel
# Validate languages against reference table
Column: First_Language_Valid
Formula: =IF(S="", "Missing",
            IF(COUNTIF(language_codes[Code], S1)=0,
            "Invalid",
            "Valid"))

Column: Second_Language_Valid
Formula: =IF(U1="", "No second language",
             IF(COUNTIF(language_codes[Code], U1)=0,
            "Invalid",
            "Valid"))

Column: Third_Language_Valid
Formula: =IF(W1="", "No third language",
            IF(COUNTIF(language_codes[Code], W1)=0,
            "Invalid",
            "Valid"))

```

Structured references did not work at this point, so direct references were used. Some languages had three letters, which weren'y included in my initial collation of country codes, like Middle English (enm) and Cebuano (ceb). One language code 'kld' did not have an online reference: 

![Language counts](../src/Screenshots/Invalid_language.png)

I check the online catalogue:

![Language counts](../src/Screenshots/Invalid_language_checke.png)

and add the correct country code to the Language Codes table.



#### 5. Download Count
Here, I will check for negative values and outliers in the set, but none are flagging.
- Negative values
- Unusually high values (outliers)

```excel
# Create download count check
Column: Download_Check
Formula: =IF(E3<0, "Invalid",
            IF(E3>AVERAGE(E3)*3, "Possible Outlier",
            "OK"))
```

#### 6. Subjects Column
Here I expect there to be multiple subjects and empty values, but I am not worried about this at the moment, as some categories


### Step 3: Data Quality Summary

As I noted in the first collection notebook, the data is incredibly high quality and there seem to be very few problematic missing elements.

![Summary sheet](../src/Screenshots/Summary_Sheet.png)

### Step 6: Final Checklist
Before considering the data clean, I have checked that:

-  All IDs are unique
-  No missing titles
-  Language codes are valid
-  Download counts are reasonable

1. Create Export View
These are the columns I will export for further analysis:
- id 
- title
- authors 
- languages 
- download_count 
- subjects
- bookshelves
- copyright
- author_count
- language_count
- first_language
- second_language
- third_language

---
**Note**: This notebook is part of the Project Gutenberg Analysis portfolio project.